In [17]:
import pandas as pd
import ray
from multiprocessing import Process, Pipe
import xgboost as xgb
import numpy as np 
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier, GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
import tensorflow
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score
import matplotlib.gridspec as gridspec
from kpal.klib import Profile
from multiprocessing import Queue
from multiprocessing import Process
from Bio.Seq import Seq
from Bio import SeqIO
from multiprocessing import Pool, freeze_support
import math
import seaborn as sns

In [18]:
cell_type = 'K562'
mod_id = 'GSM4041595_K562_rep2.hg38'
k = 4
window = 600

In [19]:
cont_data_path = f'/home/pitikovegor/CourseWork/genomes/reds_experiments/{cell_type}/all_parsed/'
sequence_path = '/home/pitikovegor/CourseWork/genomes/chr_sequence'
cluster_rna_path = '/home/pitikovegor/CourseWork/genomes/reds_experiments/K562/rna_clusters.bed'
cluster_dna_path = '/home/pitikovegor/CourseWork/genomes/reds_experiments/K562/dna_clusters.bed'
cont_wet_all_path = f'/home/pitikovegor/CourseWork/genomes/reds_experiments/{cell_type}/{cell_type}_all_contacts.tsv'
window = 500

chr_names=['chrX', 'chrY']
for i in range(1, 23):
    chr_names.append('chr%d' % (i))

In [20]:
alpha = 0.05

In [21]:
def open_table(tablename):
    #print(tablename)
    result = pd.read_csv(tablename, sep = '\t')
    #print('I read it!!!!!!!!')
    return result

In [ ]:
contacts = pd.read_csv(cont_wet_all_path, sep = '\t')

In [ ]:
contacts

In [ ]:
dna_clus = pd.read_csv(cluster_dna_path, sep = '\t', header=0, names = ['chr', 'start', 'end', 'id', 'cluster'])
cl_dna_count = pd.DataFrame(dna_clus.groupby(by = 'cluster').size()).reset_index()
display(cl_dna_count.sort_values(by = [0]))
print(cl_dna_count.loc[:, 0].quantile(1 - alpha))

In [ ]:
rna_clus = pd.read_csv(cluster_rna_path, sep = '\t', header=0, names = ['chr', 'start', 'end', 'id', 'cluster'])
cl_rna_count = pd.DataFrame(rna_clus.groupby(by = 'cluster').size()).reset_index()
display(cl_rna_count.sort_values(by = [0]))
cl_rna_count.loc[:, 0].quantile(1 - alpha)

In [ ]:
plt.rcParams['axes.titlesize'] = 8
fig = plt.figure(1, figsize=(15, 30))
gs = gridspec.GridSpec(2, 1)
ax0 = fig.add_subplot(gs[0, :])
spec = plt.hist(cl_rna_count.loc[:, 0], bins = 1000)
ax0.set_ylabel('Number')
ax0.set_title('Specific by classes', fontsize=20)
ax0.set_xticks(np.arange(2, cl_rna_count.loc[:, 0].max()))
#ax0.set_xticklabels(means.keys(), rotation=45,  ha="right", position=(0, 0))
#ax0.legend()
ax1 = fig.add_subplot(gs[1, :])
nonspec = plt.hist(cl_dna_count.loc[:, 0], bins = 1000)
#nonspec = sns.histplot(data=all_contacts, x="clus_count_rna", kde=True)
ax1.set_ylabel('Number')
ax1.set_title('Nonspecific by classes', fontsize=20)
#ax1.set_xticks(np.arange(len(means.keys())))
#ax1.set_xticklabels(means.keys(), rotation=45,  ha="right", position=(0, 0))
#ax1.legend()
#plt.subplots_adjust(wspace=0.1, hspace=1)
#plt.savefig('/Users/pitikov_egor/Documents/CourseWork/genomes/pictures/Models/%s_coninfo.png' % modelname)
plt.show()
